## 13.3 Offline Caching

W tej aplikacji zobaczymy w jaki sposób wykonać *offline caching* z wykorzystaniem lokalnej bazy danych. Zadanie polega na wykonaniu *snapshotu* danych dostępnych z zewnętrznego serwisu. Idea *offline cachingu* zakłada że urządzenie docelowe może nie mieć stałego dostępu do sieci. W przypadku braku dostępu, wykorzystujemy wersję danych dostępnych w lokalnej bazie.

W aplikacji wykorzystamy https://random-data-api.com/, który przy każdym żądaniu generuje losowe dane, wykorzystamy *endpoint* `\users` generując każdorazowo dane 20 użytkowników. W tej wersji chcemy całkowicie zastąpić istniejące dane w lokalnej bazie ich nową wersją.

Podejście to spełnia warunki *single source of truth* - pojedynczego źródła danych dostarczanych użytkownikowi. Będziemy wyświetlać tylko dane pochodzące z lokalnej bazy.

W aplikacji wykorzystamy architekturę **MVVM** wraz z bibliotekami `Dagger-Hilt`, `Retrofit`, `ROOM`, oraz `Glide`. Będzie zawierać pojedynczą aktywność na której umieścimy `RecyclerView` - tutaj nie będziemy wykorzystywać komponentu `Jetpack Navigation`.

<img src="https://media1.giphy.com/media/y0RLxRN7SPJ7mI7Hw5/giphy.gif?cid=790b7611ea236e1ec92b9220e3629a80e851a7759882cbee&rid=giphy.gif&ct=g" width="200" />

Rozpocznijmy od przygotowania modelu danych, odpowiedź serwera wygląda następująco.

In [ ]:
{
   "id":4634,
   "uid":"e1c88b6e-800c-44fe-b45a-317d6311ea32",
   "password":"rKbRvTZGIL",
   "first_name":"Scott",
   "last_name":"Veum",
   "username":"scott.veum",
   "email":"scott.veum@email.com",
   "avatar":"https://robohash.org/quianihilexercitationem.png?size=300x300\u0026set=set1",
   "gender":"Non-binary",
   "phone_number":"+420 573-352-4638 x18831",
   "social_insurance_number":"968138909",
   "date_of_birth":"1993-03-23",
   "employment":{
      "title":"Senior Agent",
      "key_skill":"Networking skills"
   },
   "address":{
      "city":"Juniorshire",
      "street_name":"Pacocha Dale",
      "street_address":"51943 Earnest Parks",
      "zip_code":"26716-5002",
      "state":"Mississippi",
      "country":"United States",
      "coordinates":{
         "lat":-56.16662202146844,
         "lng":68.36895424808517
      }
   },
   "credit_card":{
      "cc_number":"4135-2456-4633-3923"
   },
   "subscription":{
      "plan":"Premium",
      "status":"Idle",
      "payment_method":"Credit card",
      "term":"Payment in advance"
   }
}

Na jej podstawie możemy zamodelować dane - można wykorzystać dowolny dostępny plugin, czy stronę internetową.

In [ ]:
data class User(
    val address: Address,
    val avatar: String,
    val credit_card: CreditCard,
    val date_of_birth: String,
    val email: String,
    val employment: Employment,
    val first_name: String,
    val gender: String,
    val id: Int,
    val last_name: String,
    val password: String,
    val phone_number: String,
    val social_insurance_number: String,
    val subscription: Subscription,
    val uid: String,
    val username: String
)

data class Address(
    val city: String,
    val country: String,
    val state: String,
    val street_address: String,
    val street_name: String,
    val zip_code: String
)

data class CreditCard(
    val cc_number: String
)

data class Employment(
    val key_skill: String,
    val title: String
)

data class Subscription(
    val payment_method: String,
    val plan: String,
    val status: String,
    val term: String
)

Posiadając model danych, możemy przygotować layout elementu `RecyclerView` oraz głównej aktywności.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.MainActivity">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recyclerView"
        android:layout_margin="8dp"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ProgressBar
        android:id="@+id/progressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="36dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="8dp"
    android:orientation="vertical">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <TextView
            android:id="@+id/firstName"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="1"
            android:text="TextView"
            android:textSize="24sp"
            app:layout_constraintStart_toStartOf="parent"
            tools:layout_editor_absoluteY="22dp" />

        <TextView
            android:id="@+id/lastName"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_weight="1"
            android:text="TextView"
            android:textSize="24sp"
            app:layout_constraintEnd_toEndOf="parent"
            tools:layout_editor_absoluteY="20dp" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:orientation="vertical">

            <TextView
                android:id="@+id/username"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/password"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/email"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/gender"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/phone"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/dateOfBirth"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />

            <TextView
                android:id="@+id/employment"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:text="TextView" />


        </LinearLayout>

        <ImageView
            android:id="@+id/image"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:src="@drawable/ic_android_black_24dp"
            android:layout_height="wrap_content"
            android:contentDescription="user image" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="8dp"
        android:orientation="horizontal">

        <TextView
            android:id="@+id/country"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/city"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/state"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />
    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="8dp"
        android:orientation="horizontal">

        <TextView
            android:id="@+id/street_name"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/street_address"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/zip_code"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />
    </LinearLayout>

    <TextView
        android:id="@+id/creditCardNumber"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="TextView" />

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="8dp"
        android:orientation="horizontal">

        <TextView
            android:id="@+id/plan"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/status"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/payment_method"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />

        <TextView
            android:id="@+id/term"
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:text="TextView" />
    </LinearLayout>
</LinearLayout>

Dodajmy interfejs `RandomApi` z jedną metodą, która zwraca listę 20 użytkowników

In [ ]:
interface RandomApi {
    @GET("users?size=20")
    suspend fun users(): List<User>
}

Dodajmy `AppModule` z metodą dostarczającą instancję obiektu o typie `RandomApi`.

In [ ]:
@Module
@InstallIn(SingletonComponent::class)
object AppModule {

    @Provides
    @Singleton
    fun provideRandomApi(): RandomApi{
        val interceptor = HttpLoggingInterceptor()
        interceptor.setLevel(HttpLoggingInterceptor.Level.BODY)
        val client = OkHttpClient.Builder()
            .addInterceptor(interceptor)
            .build()
        return Retrofit.Builder()
            .baseUrl("https://random-data-api.com/api/v2/")
            .addConverterFactory(GsonConverterFactory.create())
            .client(client)
            .build().create(RandomApi::class.java)
    }
}

Przejdźmy do utworzenia lokalnej bazy danych - tutaj od razu napotykamy problem. Bazy danych mogą przyjmować tylko określone typy, nie mogą przechowywać obiektów niestandardowych. Klasa `User` posiada obiekty o typach `Address`, `CreditCard`, `Employment` i `Subsciption`. Aby zapisać instancje tych obiektów w bazie `ROOM`, musimy je przekonwertować do typu standardowego - tutaj wyborem jest `String`.

Zaimplementujemy kilka obiektów - dla każdego typu - opisujących sposób konwersji. Rozpocznijmy od `AddressConverter`.

In [ ]:
object AddressConverter {}

Obiekt będzie posiadał dwie metody - dla konwersji `Address` -> `String`, oraz dla konwersji `String` -> `Address`.

Samą konwersję można przeprowadzić na szereg różnych sposobów - tutaj wykorzystamy metodę `toString` (wszystkie klasy są klasami danych, więc posiadają domyślną implementację tej metody).

In [ ]:
@TypeConverter
@JvmStatic
fun fromAddress (address: Address): String{
    return address.toString()
}

Stosujemy dwie adnotacje
- `@TypeConverter` - adnotacja oznacza metody wykorzystywane do konwersji - samą konwersję `ROOM` przeprowadza automatycznie
- `@JvmStatic` - musimy tylko dostarczyć metody konwersji, nie obiekt z nimi powiązany (stąd wykorzystanie `object` a nie `class`) - dzięki tej adnotacji zostanie utworzona metoda statyczna w obiekcie.

Potrzebujemy jeszcze drugą metodę konwersji - tutaj nie ma pojedynczej metody do rozpakowania `String`, który otrzymujemy, więc zrobimy to wykorzystując metody `substring`

In [ ]:
@TypeConverter
@JvmStatic
fun toAddress(address: String): Address{
    val city = address.substringAfter("city=").substringBefore(",")
    val country = address.substringAfter("country=").substringBefore(",")
    val state = address.substringAfter("state=").substringBefore(",")
    val street_address = address.substringAfter("street_address=").substringBefore(",")
    val street_name = address.substringAfter("street_name=").substringBefore(",")
    val zip_code = address.substringAfter("zip_code=").substringBefore(")")

    return Address(city, country, state, street_address, street_name, zip_code)
}

Nie jest to typowa implementacja - często konwersja jest wykonywana do i z formatu `JSON` wykorzystując konwerter `Gson`

Dodajmy pozostałe obiekty konwerterów.

In [ ]:
object CreditCardConverter {
    @TypeConverter
    @JvmStatic
    fun fromCreditCard (creditCard: CreditCard): String{
        return creditCard.cc_number
    }

    @TypeConverter
    @JvmStatic
    fun toCreditCard(creditCard: String): CreditCard {
        return CreditCard(creditCard)
    }
}

object EmploymentConverter {

    @TypeConverter
    @JvmStatic
    fun fromEmployment (employment: Employment): String{
        return employment.title
    }

    @TypeConverter
    @JvmStatic
    fun toEmployment(employment: String): Employment {
        return Employment(employment, employment)
    }
}

object SubscriptionConverter {

    @TypeConverter
    @JvmStatic
    fun fromSubscription (subscription: Subscription): String{
        return subscription.toString()
    }

    @TypeConverter
    @JvmStatic
    fun toSubscription(subscription: String): Subscription {
        val payment_method = 
            subscription.substringAfter("payment_method=").substringBefore(",")
        val plan = subscription.substringAfter("plan=").substringBefore(",")
        val status = subscription.substringAfter("status=").substringBefore(",")
        val term = subscription.substringAfter("term=").substringBefore(")")

        return Subscription(payment_method, plan, status, term)
    }
}

Przejdźmy do implementacji samej bazy, oznaczmy klasę `User` jako `@Entity` oraz `id` jako `@PrimaryKey`.

In [ ]:
@Entity(tableName = "users")
data class User(
    ...
    @PrimaryKey val id: Int,
    ...
    )

Zdefiniujmy `Dao` z trzema metodami

In [ ]:
@Dao
interface UsersDao {
    @Insert(onConflict = REPLACE)
    suspend fun insert(user: List<User>)

    @Query("DELETE FROM users")
    suspend fun clear()

    @Query("SELECT * FROM users")
    fun getUsers(): Flow<List<User>>
}

Ponieważ chcemy zastępować całą listę w bazie, potrzebujemy dwóch metod
- `clear` - czyści bazę
- `instert` - dodaje nową listę

Mamy również metodę zwracającą całą listę. Zwróćmy uwagę na zwracany typ. `Flow` jest asynchronicznym strumieniem wartości - oznacza to że zamiast jednej listy użytkowników, jest ona emitowana - co znacza że jest ona obserwowalna. Funkcja `getUsers` zwróci nam listę wszystkich użytkowników, gdy zmieni się jakakolwiek wartość w tabeli `users`, `ROOM` automatycznie wyemituje nową wersję listy użytkowników. Gdy lokalna baza zostanie zaktualizowana o nowe dane w serwera, `ROOM` wyemituje nową wersję, dzięki czemu możemy automatycznie dostarczyć nowe dane użytkownikowi.

Dodajmy klasę abstrakcyjną `UserDatabase`, wykorzystamy adnotację `TypeConverters` do zakomunikowania `ROOM` że wymagane jest wykorzystanie konwerterów, oraz podamy jawnie wszystkie obiekty konwerterów.

In [ ]:
@Database(entities = [User::class], version = 1)
@TypeConverters(
    AddressConverter::class, 
    CreditCardConverter::class, 
    EmploymentConverter::class, 
    SubscriptionConverter::class)
abstract class UserDatabase : RoomDatabase() {

    abstract fun usersDao(): UsersDao
}

Do  klasy `AppModule` dodajmy metodę dostarczającą bazę danych

In [ ]:
@Provides
@Singleton
fun provideUserDatabase(app: Application): UserDatabase = 
    Room
        .databaseBuilder(
            app, 
            UserDatabase::class.java, 
            "kotlin_user_database")
        .build()

Dodajmy klasę `Resources` znaną z poprzendnich przykładów

In [ ]:
sealed class Resource<T>(
    val data: T? = null,
    val throwable: Throwable? = null
) {
    class Success<T>(data: T) : Resource<T>(data)
    class Error<T>(throwable: Throwable, data: T? = null) : Resource<T>(data, throwable)
    class Loading<T>(data: T? = null) : Resource<T>(data)
}

Zdefiniujmy funkcję `networkBoundResource` w pliku `NetworkBoundResource`, bedzie ona odpowiedzialna za logikę dostępu do danych - kiedy i w jakich warunkach dostajemy dane z serwera lub bazy lokalnej.

In [ ]:
inline fun <ResultType, RequestType> networkBoundResource()

`inline` poprawia znacznie wydajność tej funkcji. Funkcja będzie przyjmować funkcje jako argumenty i koordynować ich pracę - czyli okreslić kiedy wykonać połączenie z bazą lokalną, kiedy z api. Do funkcji przekazujemy dwa generyczne argumenty `ResultType` i `RequestType` - funkcja posiada dwa argumenty, ponieważ typ danych otrzymanych z api może się różnić od typu otrzymanego z lokalnej bazy - w tej aplikacji typ jest taki sam.

Jako argumenty przekażemy cztery funkcje
- query: () -> Flow<ResultType> - odpowiedzialna za pobranie danych z lokalnej bazy danych - nie przyjmuje argumentów, zwraca `Flow<ResultType>` - tutaj `ResultType` będzie odpowiadał `List<User>`.
- fetch: suspend () -> RequestType - funkcja z zawieszeniem wykonania, nie przyjmująca argumentów, odpowiedzialna za pobranie danych z api, zwraca `RequestType` - tutaj `RequestType` odpowieda `List<User>`.
- saveFetchResult: suspend (RequestType) -> Unit - funkcja z zawieszeniem wykonania, odpowiedzialna za przejęcie i zapisanie danych z api do bazy lokalnej, przyjmuje jeden argument - w naszej aplikacji bedzie przyjmować listę użytkowników pobraną z api.
- shouldFetch: (ResultType) -> Boolean = {true} - decyduje czy dane z lokalnej bazy są aktualne i czy należy wykonać aktualizację, przejmuje jeden argument typu `ResultType` i zwraca `Booldean` - domyślnie wartość jest ustawiona na `True`.

Popnieważ nasza funkcja posiada modyfikator `inline`, argumenty zwracające wartość muszą być oznaczone przez `crossinline`.

In [ ]:
inline fun <ResultType, RequestType> networkBoundResource(
    crossinline query: () -> Flow<ResultType>,
    crossinline fetch: suspend () -> RequestType,
    crossinline saveFetchResult: suspend (RequestType) -> Unit,
    crossinline shouldFetch: (ResultType) -> Boolean = {true} // domyślnie zawsze aktualizuje
)

funkcja tworzy *zimny* `Flow` - zimny strumień (`Flow`) nie emituje danych dopóki inny obiekt nie rozpocznie odbierania.

In [ ]:
inline fun <ResultType, RequestType> networkBoundResource(
    crossinline query: () -> Flow<ResultType>,
    crossinline fetch: suspend () -> RequestType,
    crossinline saveFetchResult: suspend (RequestType) -> Unit,
    crossinline shouldFetch: (ResultType) -> Boolean = {true}
) = flow {}

W pierwszej kolejności musimy zdecydować czy chcemy dostać nową wersję danych z api, czy nie. Utwórzmy dane

In [ ]:
val data = query().first()

Wywołujemy funkcję `query`, ponieważ zwraca ona `Flow`, a tutaj potrzebujemy tylko jedną listę - wywaołujemy funkcję `first`.

Następnie w warunku `if` sprawdzamy czy musimy pobrać nowe dane

if (shouldFetch(data)){

Jako argument podajemy dane, które wcześniej dostaliśmy z lokalnej bazy.

Jeżeli musimy pobrać nowe dane, emitujemy stan ładowania (`Resource.Loading`)

In [ ]:
emit(Resource.Loading(data))

Możemy wyemitować wartości, wywołując funkcję `emit` - dzięki temu możemy wyświetlić na ekranie dane z bazy lokalnej podczas ładowania danych z api.

Nasrtępnie chcemy pobrać dane z api i dodać je do bazy lokalnej.

In [ ]:
saveFetchResult(fetch())

Musimy obsłużyć sytuację w której nasze żądanie nie zostanie zakończone sukcesem. Dodajmy blok `try-catch`

In [ ]:
try {
    saveFetchResult(fetch())
} catch (throwable: Throwable){

}

Jeżeli żądanie zostało zakończone sukcesem, chcemy przekazać dane do ui i wyemitować stan `Resource.Success`

In [ ]:
try {
    saveFetchResult(fetch())
    query().map { Resource.Success(it) } // zwraca Flow<Resource<List<User>>>
} catch (throwable: Throwable){

}

Jeżeli dostaniemy błąd, chcemy wykonać aktualizację lokalną (korzystając z bazy) i wyemitować stan `Resource.Error`

In [ ]:
try {
    saveFetchResult(fetch())
    query().map { Resource.Success(it) } // zwraca Flow<Resource<List<User>>>
} catch (throwable: Throwable){
    query().map { Resource.Error(throwable, it) }
}

Jeżeli `if (shouldFetch(data))` zwróci `false`, chcemy zwrócić aktualne dane z bazy, niewymagające aktualizacji. 

In [ ]:
if (shouldFetch(data)){
    emit(Resource.Loading(data))
    try {
        saveFetchResult(fetch())
        query().map { Resource.Success(it) }
    } catch (throwable: Throwable){
        query().map { Resource.Error(throwable, it) }
    }
} else query().map { Resource.Success(it) }

Ponieważ `query.map()` zwraca `Flow`, możemy utworzyć wartość i ją wyemitować.

In [ ]:
val flow = if (shouldFetch(data)){
    emit(Resource.Loading(data))
    try {
        saveFetchResult(fetch())
        query().map { Resource.Success(it) }
    } catch (throwable: Throwable){
        query().map { Resource.Error(throwable, it) }
    }
} else query().map { Resource.Success(it) }

emitAll(flow) // emituje przyszłe aktualizacje bazy

Pełny kod `networkBoundResource`

In [ ]:
inline fun <ResultType, RequestType> networkBoundResource(
    crossinline query: () -> Flow<ResultType>,
    crossinline fetch: suspend () -> RequestType,
    crossinline saveFetchResult: suspend (RequestType) -> Unit,
    crossinline shouldFetch: (ResultType) -> Boolean = {true}
) = flow {
    val data = query().first()
    val flow = if (shouldFetch(data)){
        emit(Resource.Loading(data))
        try {
            saveFetchResult(fetch())
            query().map { Resource.Success(it) }
        } catch (throwable: Throwable){
            query().map { Resource.Error(throwable, it) }
        }
    } else query().map { Resource.Success(it) }

    emitAll(flow)
}

Dodajmy repozytorium, wykorzystamy wstrzyknięcie przez konstruktor `RandomApi` oraz `UserDatabase`

In [ ]:
class UserRepository @Inject constructor (
    private val api: RandomApi,
    private val db: UserDatabase
        ) {

W repozytorium zdefiniujmey jedną funkcję `getUsers`, w której wywołamy poprzednio zaimplementowaną `networkBoundResource`

In [ ]:
fun getUsers() = networkBoundResource(

Jako pierwszy argument przekazujemy funkcję `getUsers` z naszego `Dao`.

In [ ]:
query = { db.usersDao().getUsers() },

Jako drugi argument przekazujemy funkcję `users` z `RandomApi`

In [ ]:
fetch = { api.users() },

Funkcję `saveFetchResult` wywołamy jako `withTransaction`

In [ ]:
saveFetchResult = { users ->
    db.withTransaction {

`withTransaction` oznacza że **wszystkie** operacje w tym bloku będą wykonane lub **żadna**.

In [ ]:
saveFetchResult = { users ->
    db.withTransaction {
        db.usersDao().apply {
            clear()
            insert(users)
        }
    }
}

Ponieważ wywołujemy dwie funkcje: `clear` - czyszcząca bazę, oraz `insert` - dodająca całą listę, wykorzystanie `withTransaction` pozwala nam uniknąć sytuacji w której usuwamy całą bazę, a `insert` nie kończy się sukcesem.

Czwarty argument w tym przykładzie pozostawiamy z wartością domyślną.

Pełny kods repozytorium:

In [ ]:
class UserRepository @Inject constructor (
    private val api: RandomApi,
    private val db: UserDatabase
        ) {
    fun getUsers() = networkBoundResource(
        query = { db.usersDao().getUsers() },
        fetch = { api.users() },
        saveFetchResult = { users ->
            db.withTransaction {
                db.usersDao().apply {
                    clear()
                    insert(users)
                }
            }
        }
    )
}

Dodajmy `ViewModel

In [ ]:
@HiltViewModel
class UserViewModel @Inject constructor(
    repository: UserRepository
) : ViewModel() {
    val users = repository.getUsers().asLiveData()
}

Metoda `getUsers` zwraca `Flow`, więc wykorzystujemy `asLiveData` aby dostać instancję `LiveData`

W aktywności fgłównej posiadamy `RecyclerView`, więc dodajmyn `Comparator`, `ViewHolder` oraz `Adapter`

In [ ]:
class UserComparator : DiffUtil.ItemCallback<User>() {
    override fun areItemsTheSame(oldItem: User, newItem: User): Boolean {
        return newItem.uid == oldItem.uid
    }

    override fun areContentsTheSame(oldItem: User, newItem: User): Boolean {
        return newItem == oldItem
    }
}

class UserViewHolder(private val binding: RvItemBinding) : 
    RecyclerView.ViewHolder(binding.root) {
    fun bind(item: User){
        binding.firstName.text = item.first_name
        binding.lastName.text = item.last_name
        binding.city.text = item.address.city
        binding.country.text = item.address.country
        binding.dateOfBirth.text = item.date_of_birth
        binding.creditCardNumber.text = item.credit_card.cc_number
        binding.email.text = item.email
        binding.employment.text = item.employment.title
        binding.gender.text = item.gender
        binding.password.text = item.password
        binding.paymentMethod.text = item.subscription.payment_method
        binding.phone.text = item.phone_number
        binding.plan.text = item.subscription.plan
        binding.term.text = item.subscription.term
        binding.paymentMethod.text = item.subscription.payment_method
        binding.state.text = item.address.state
        binding.zipCode.text = item.address.zip_code
        binding.status.text = item.subscription.status
        binding.streetAddress.text = item.address.street_address
        binding.streetName.text = item.address.street_name
        binding.username.text = item.username
        Glide.with(binding.root).load(item.avatar).into(binding.image)
    }
}
    
class UserAdapter(comparator: UserComparator) : 
    ListAdapter<User, UserViewHolder>(comparator) {
    override fun onCreateViewHolder(
        parent: ViewGroup, 
        viewType: Int
    ): UserViewHolder = UserViewHolder(
            RvItemBinding.inflate(
                LayoutInflater.from(parent.context), parent, false
            )
        )

    override fun onBindViewHolder(
        holder: UserViewHolder, 
        position: Int
    ) = holder.bind(getItem(position))
}

Dodajmy klasę `Application`

In [ ]:
@HiltAndroidApp
class UsersApplication : Application() {
}

Na koniec zaimplementujmy niezbędne elementy w aktywności głównej

In [ ]:
@AndroidEntryPoint
class MainActivity : AppCompatActivity() {

    private val binding: ActivityMainBinding by lazy {
        ActivityMainBinding.inflate(layoutInflater)
    }

    private val viewModel: UserViewModel by viewModels()

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(binding.root)

        val adapter = UserAdapter(UserComparator())
        setupRecyclerView(adapter)

        viewModel.users.observe(this) { result ->
            adapter.submitList(result.data)

            binding.progressBar.isVisible = 
                result is Resource.Loading && result.data.isNullOrEmpty()
        }

    }

    private fun setupRecyclerView(userAdapter: UserAdapter) {
        binding.recyclerView.apply {
            adapter = userAdapter
            layoutManager = LinearLayoutManager(this@MainActivity)
        }
    }
}

Możemy przetestować aplikację. Jak widzimy przy pierwszym uruchomieniu ładowane są dane, przy drugim wyświetlane są dane *offline cached* z bazy lokalnej, asynchronicznie wykonywana jest aktualizacja z serwera.

<img src="https://media1.giphy.com/media/y0RLxRN7SPJ7mI7Hw5/giphy.gif?cid=790b7611ea236e1ec92b9220e3629a80e851a7759882cbee&rid=giphy.gif&ct=g" width="150" />